In [1]:
import os
os.chdir('..')

In [2]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt

import Yukawa_SINDy as ys
import cross_validation as cv

from importlib import reload
import pickle as pkl
with open('scaling_const.float', 'rb') as f:
    SCALING_CONST = pkl.load(f)

/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/Yukawa_SINDy.py:674: SyntaxWarning: invalid escape sequence '\o'
  ax.set_title("simulation duration: " + str(sim_duration) + "$\omega_{pd}$")
/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/Yukawa_SINDy.py:677: SyntaxWarning: invalid escape sequence '\l'
  ax.set_ylabel("max ptcl sep ($\lambda_{De}$)")
/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/Yukawa_SINDy.py:691: SyntaxWarning: invalid escape sequence '\l'
  axs[i].set_title("Cutoff: " + str(cutoffs[i]) + "$\lambda_{De}$", fontsize=14)
/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/Yukawa_SINDy.py:693: SyntaxWarning: invalid escape sequence '\o'
  fig.supxlabel('duration ($\omega_{pd}^{-1}$)')
/Users/zbh0005/Library/CloudStorage/OneDrive-AuburnUniversity/Documents/Code/yukawa-sindy/Yukawa_SINDy.py:695: SyntaxWarnin

# Investigation of noise robustness of the Weak SINDy method with cross-validation